In [7]:
!pip install ngrok
!pip install flask-cors
!pip install pyngrok
!pip install pdfplumber
!pip install ftfy
!pip install scikit-learn

In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_d

True

In [13]:
import re
import json
import pdfplumber
import ftfy
from transformers import pipeline
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import unicodedata
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK sentence tokenizer
from nltk.tokenize import sent_tokenize

# Initialize the model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def summarize_with_embeddings(text):
    """Generate embeddings for each sentence and return the most representative ones."""
    # Tokenize sentences properly
    sentences = sent_tokenize(text)

    # Generate embeddings for each sentence
    sentence_embeddings = model.encode(sentences)

    # Compute the overall document embedding as the mean of all sentence embeddings
    doc_embedding = np.mean(sentence_embeddings, axis=0).reshape(1, -1)

    # Compute cosine similarity between each sentence embedding and the document embedding
    similarities = cosine_similarity(sentence_embeddings, doc_embedding).flatten()

    # Select the top 2 most representative sentences
    top_indices = similarities.argsort()[-2:][::-1]
    summary = ' '.join([sentences[i] for i in top_indices])

    return summary

def extract_tech_name(Field_Name):
    # Original string
    text = Field_Name
    extracted_text = ""
    # Regular expression to match text inside parentheses
    matches = re.findall(r'\((.*?)\)', Field_Name, re.DOTALL)

    # Check if a match is found and print the result
    if matches:
        extracted_text = matches[-1].replace("\n", "")
        #print(extracted_text)
    else:
        print("No match found")

    cleaned_text = extracted_text.replace(" ", "")
    return cleaned_text

def clean_newline(text_field):
    # Original string
    text = text_field

    cleaned_text = text.replace("\n", " ")
    return cleaned_text

def validate_psrlei(psr, obligor, lei):
    """
    Validate Primary Source of Repayment LEI (PSRLEI) based on FRY14Q rules using LLM.
    :param psr: Primary Source of Repayment (Field 50)
    :param obligor: Obligor (Field 2 or 4)
    :param lei: LEI provided (Field 112)
    :return: Validation result (Valid or Error Message)
    """
    generator = pipeline("text-generation", model="gpt-3.5-turbo")

    prompt = f"""
    Given the following inputs:
    - Primary Source of Repayment (PSR): {psr}
    - Obligor: {obligor}
    - LEI: {lei}

    Apply the following validation rules:
    1. If PSR = Obligor, LEI must be blank.
    2. If PSR ≠ Obligor, LEI must be a valid 20-character alphanumeric code or 'NA'.
    3. If LEI is invalid, return an appropriate error message.

    Return 'Valid' if all rules are met, otherwise return the specific error message.
    """

    result = generator(prompt, max_length=100, do_sample=False)[0]['generated_text']
    return result

def extract_tables_from_pdf(pdf_path, schedule="H.1 - Corporate Loan Data Schedule", start_text="H.1 - Corporate Loan Data Schedule", end_text="H.2 – Commercial Real Estate Schedule"):

    """Extracts tables only from the specified schedule (H.1) between start and end markers."""
    table_data = []
    with pdfplumber.open(pdf_path) as pdf:
        extract = False
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                if start_text in text:
                    extract = True
                if extract:
                    tables = page.extract_table()
                    if tables:
                        table_data.extend(tables)  # Collect all table rows
                if end_text in text:
                    extract = False
                    break
    return table_data


def clean_text(text):
    """Cleans extracted text by removing backslashes, forward slashes, and extra spaces."""
    if text:
        return re.sub(r"[\\/]+", "", text).strip()
    return ""

def process_table_data(table_data):
    """Processes extracted table data into structured fields with complete descriptions."""
    # print(f"Number of rows in the table data: {len(table_data)}")
    fields = {}
    i=0
    for row in table_data:
        i=i+1
        if len(row) <= 5:  # Ensuring the row has enough columns
            field_no, field_name, technical_name, description, constraints = map(clean_text, row[:5])

            if field_name and technical_name:  # Ensure valid field entries

                if field_name in fields:  # If field already exists, append to description (handles multi-line cases)
                    fields[field_name]["description"] += " " + description
                else:
                    fields[field_name] = {
                        "field_no": field_no,
                        "field_name": clean_newline(field_name),
                        "technical_name": technical_name,
                        "description": clean_newline(unicodedata.normalize("NFKC", re.sub(r'\s+', ' ', bytes(ftfy.fix_text(description), "utf-8").decode("utf-8", "ignore")).strip())),
                        "constraints": clean_newline(unicodedata.normalize("NFKC", re.sub(r'\s+', ' ', bytes(ftfy.fix_text(constraints), "utf-8").decode("utf-8", "ignore")).strip())),
                         "mandatory": True,  # Assuming all fields are mandatory
                    }
    return fields

def generate_rule_dictionary(fields):
    """Generates validation rules dynamically based on extracted fields."""
    rule_dict = {}
    # print(f"Length of fields: {len(fields)}")
    for field, properties in fields.items():
        # print("Field No, FieldName")
        # print(properties["field_no"],properties["field_name"])

        rule_dict[field] = {
            "field_no": properties["field_no"],
            "field_name": properties["field_name"],
            "technical_name": extract_tech_name(properties["field_name"]),
            "description": properties["description"],
            "allowable_values": properties["constraints"],
            "validation_rules": summarize_with_embeddings(properties["description"])+"\n"+properties["constraints"]

        }
    return rule_dict

In [24]:
import json
import nltk
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load JSON file with rules
def load_rules(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

rules_data = load_rules("FRY14QH1_rules.txt")

# Extract validation rules into a DataFrame
rules = []
for field, details in rules_data["Corporate Loan Data Fields"].items():
    rules.append({
        "Field": field,
        "Condition": details["validation"],
        "Valid": 1,
        "Valid_Example": details["valid_example"],
        "Invalid_Example": details["invalid_example"]
    })

df = pd.DataFrame(rules)

def extract_features(text, valid_example, invalid_example):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return {
        "num_tokens": len(tokens),
        "num_nouns": len([word for word, pos in pos_tags if pos.startswith("NN")]),
        "num_verbs": len([word for word, pos in pos_tags if pos.startswith("VB")]),
        "valid_example_length": len(valid_example) if isinstance(valid_example, str) else 0,
        "invalid_example_count": len(invalid_example) if isinstance(invalid_example, list) else 0
    }

features_df = pd.DataFrame(df.apply(lambda row: extract_features(str(row['Condition']), row['Valid_Example'], row['Invalid_Example']), axis=1).tolist())
X_train, X_test, y_train, y_test = train_test_split(features_df, df['Valid'], test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Function to validate individual field values
def validate_input(field_name, value, internal_id=None):
    errors = []
    field_info = rules_data["Corporate Loan Data Fields"].get(field_name, {})

    if not field_info:
        return "Error: No validation rules found."

    validation_rule = field_info.get("validation", "")
    valid_example = field_info.get("valid_example", "No example available.")
    invalid_example = field_info.get("invalid_example", ["No invalid examples."])

    if not value or pd.isna(value):
        errors.append(f"'{field_name}' cannot be empty.")

    value = str(value).strip()

    if "must be a whole number" in validation_rule.lower() and not re.fullmatch(r"^-?\d+$", value):
        errors.append(f"'{value}' must be a valid whole number.")
    if "YYYY-MM-DD" in validation_rule and not re.fullmatch(r"^\d{4}-\d{2}-\d{2}$", value):
        errors.append(f"'{value}' must be in YYYY-MM-DD format.")
    if "must be uppercase" in validation_rule.lower() and not value.isupper():
        errors.append(f"'{value}' must be uppercase.")
    if "max 20 characters" in validation_rule.lower() and len(value) > 20:
        errors.append(f"'{value}' exceeds the max length of 20 characters.")
    if "must be uppercase" in validation_rule.lower() and not value.isupper():
        errors.append(f"'{value}' must be uppercase.")

    if "must be exactly 2 characters" in validation_rule.lower() and not re.fullmatch(r"^[A-Z]{2}$", value):
        errors.append(f"'{value}' must be exactly 2 characters (ISO 3166-1 alpha-2 country code).")

    if "YYYY-MM-DD" in validation_rule and not re.fullmatch(r"^\d{4}-\d{2}-\d{2}$", value):
        errors.append(f"'{value}' must be in YYYY-MM-DD format.")

    if "must be a future date" in validation_rule and value <= "2024-03-26":
        errors.append(f"'{value}' must be a future date.")

    if "must be a past or present date" in validation_rule and value > "2024-03-26":
        errors.append(f"'{value}' must be a past or present date.")

    if "decimal format" in validation_rule.lower() and not re.fullmatch(r"^\d+(\.\d+)?$", value):
        errors.append(f"'{value}' must be a valid decimal number.")

    if "percentage format" in validation_rule.lower() and not re.fullmatch(r"^\d{1,3}(\.\d{1,2})?$", value):
        errors.append(f"'{value}' must be a valid percentage (0-100, max 2 decimals).")

    if field_name == "Country" and not re.fullmatch(r"^[A-Z]{2}$", value):
        errors.append(f"'{value}' must be a valid 2-letter country code (ISO 3166-1). Example: 'US'.")

    if "must be alphanumeric" in validation_rule.lower() and not re.fullmatch(r"^[a-zA-Z0-9]+$", value):
        errors.append(f"'{value}' must be alphanumeric, with no special characters.")

    if "max 20 characters" in validation_rule.lower() and len(value) > 20:
        errors.append(f"'{value}' exceeds the maximum length of 20 characters.")

    if "must match format of Internal_ID" in validation_rule.lower() and internal_id is not None:
        if not re.fullmatch(r"^[a-zA-Z0-9]+$", value) or len(value) != len(internal_id):
            errors.append(f"'{value}' must match the format and length of Internal_ID '{internal_id}'.")
    if "cannot contain spaces or special characters" in validation_rule.lower() and not re.fullmatch(r"^[a-zA-Z0-9]+$", value):
        errors.append(f"'{value}' cannot contain spaces or special characters.")
    if "max 50 characters" in validation_rule.lower() and len(value) > 50:
        errors.append(f"'{value}' exceeds the maximum length of 50 characters.")
    if "max 10 characters" in validation_rule.lower() and len(value) > 10:
        errors.append(f"'{value}' exceeds the maximum length of 10 characters.")
    if "max 100 characters" in validation_rule.lower() and len(value) > 100:
        errors.append(f"'{value}' exceeds the maximum length of 100 characters.")
    if "must be a whole number" in validation_rule.lower():
        if not re.fullmatch(r"^\d+$", value):
            errors.append(f"'{value}' must be a whole number with no decimals, commas, or symbols.")
        elif int(value) < 0:
            errors.append(f"'{value}' cannot be negative.")
    if "decimal format" in validation_rule.lower():
        if not re.fullmatch(r"^0(\.\d{1,7})?$|^1(\.0{1,7})?$", value):
            errors.append(f"'{value}' must be a decimal number between 0 and 1 with up to 7 decimal places.")
        elif not (0 <= float(value) <= 1):
            errors.append(f"'{value}' must be between 0 and 1.")
    if "decimal format" in validation_rule.lower():
        if not re.fullmatch(r"^\d{1,3}(\.\d{1,4})?$", value):
            errors.append(f"'{value}' must be a decimal number with up to 4 decimal places, between 0 and 100.")
        elif not (0 <= float(value) <= 100):
            errors.append(f"'{value}' must be between 0 and 100.")
    if field_name == "Currency" and not re.fullmatch(r"^[A-Z]{3}$", value):
        errors.append(f"'{value}' must be a valid ISO 4217 three-letter currency code in uppercase.")
    if field_name == "Currency" and not re.fullmatch(r"^[A-Z]{3}$", value):
        errors.append(f"'{value}' must be a valid ISO 4217 three-letter currency code in uppercase.")
    if "enumerated values" in validation_rule.lower():
            valid_options = re.findall(r"\((.*?)\)", validation_rule)
            if valid_options:
                valid_options = [opt.strip() for opt in valid_options[0].split(",")]
                if value not in valid_options:
                    errors.append(f"'{value}' is not a valid option. Allowed values: {', '.join(valid_options)}.")
    if errors:
        return "Non-Compliant", errors

    # Machine Learning validation
    input_features = extract_features(validation_rule, valid_example, invalid_example)
    input_df = pd.DataFrame([input_features])
    prediction = clf.predict(input_df)[0]

    return ("Compliant" if prediction == 1 else "Non-Compliant (ML Prediction)", [])

# Function to validate an entire XLS file
def validate_xls(file_path):
    df = pd.read_excel(file_path)
    anomalies = {col: [] for col in df.columns}

    for _, row in df.iterrows():
        for field_name, value in row.items():
            compliance_status, issues = validate_input(field_name, value)
            anomalies[field_name].append((value, compliance_status, "; ".join(issues)))

    anomalies_df = pd.DataFrame({col: [list(x) for x in zip(*values)] for col, values in anomalies.items()},
                                index=["Value", "Compliance_Status", "Issues"]).T

    # anomalies_df.to_excel("anomalies_report.xlsx", index=True)
    # anomalies_df.to_json("anomalies_report.json", orient="records", indent=4)
    json_data = anomalies_df.reset_index().to_json(orient="records", indent=4)
    return json_data


[
    {
        "index":"Customer_ID",
        "Value":[
            "123456789012345ABCD"
        ],
        "Compliance_Status":[
            "Compliant"
        ],
        "Issues":[
            ""
        ]
    },
    {
        "index":"Internal_ID",
        "Value":[
            "4f762d87-5c8a-430a-a18a-e9b9c1b7073f"
        ],
        "Compliance_Status":[
            "Non-Compliant"
        ],
        "Issues":[
            "'4f762d87-5c8a-430a-a18a-e9b9c1b7073f' exceeds the max length of 20 characters.; '4f762d87-5c8a-430a-a18a-e9b9c1b7073f' must be alphanumeric, with no special characters.; '4f762d87-5c8a-430a-a18a-e9b9c1b7073f' exceeds the maximum length of 20 characters."
        ]
    },
    {
        "index":"Original_Internal_ID",
        "Value":[
            "62956d9f-0445-4566-86ac-960cdc8db76b"
        ],
        "Compliance_Status":[
            "Non-Compliant"
        ],
        "Issues":[
            "'62956d9f-0445-4566-86ac-960cdc8db76b' cannot contain spaces or 

In [ ]:
from flask import Flask, request, jsonify
import requests
import torch
from pyngrok import ngrok
import base64
from flask_cors import CORS
import json
import os

app = Flask(__name__)
CORS(app)

# Load rules.json and send it as a response
@app.route('/get_current_rules', methods=['GET'])
def get_current_rules():
    try:
        with open("FRY14QH1_rules.json", "r", encoding="utf-8") as file:
            rules_data = json.load(file)
        return jsonify(rules_data), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# API to handle PDF file upload
@app.route('/generate_new_rules', methods=['POST'])
def upload_pdf():
    try:
        UPLOAD_FOLDER = "pdfUploads"
        os.makedirs(UPLOAD_FOLDER, exist_ok=True)
        file = request.files['file']
        # Save PDF file
        pdf_path = os.path.join(UPLOAD_FOLDER, "generate_rules.pdf")
        file.save(pdf_path)
        print(f"Received PDF: {file.filename}")

        pdf_table_data = extract_tables_from_pdf(pdf_path, schedule="H.1 - Corporate Loan Data Schedule", start_text="H.1 - Corporate Loan Data Schedule", end_text="H.2 – Commercial Real Estate Schedule")
        extracted_fields = process_table_data(pdf_table_data)
        rule_dictionary = generate_rule_dictionary(extracted_fields)
        json_text = json.dumps(rule_dictionary, indent=4)

        file_path = "FRY14QH1_rules.json"
        with open(file_path, "w") as json_file:
            json.dump(json_text, json_file, indent=4)

        print(f"JSON data has been written to {file_path}")
        return jsonify({'status': 'success', 'message': 'Rules are successfully generated.'}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# API to generate anomalies report
@app.route('/generate_anomalies_report', methods=['POST'])
def upload_excel():
    try:
        UPLOAD_FOLDER = "sampleDataUploads"
        os.makedirs(UPLOAD_FOLDER, exist_ok=True)
        file = request.files["file"]
        file_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(file_path)
        df = pd.read_excel(file_path)
        json_data = validate_xls(file_path)

        return jsonify({"status": "success", "data": json_data}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    # Set ngrok auth token
    ngrok.set_auth_token("2UCBnO3oWGPJqVUOcUQ1yDWtUCY_4eizetwxTW6ZoesLiA2wG")
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print('Public URL:', public_url)
    app.run(host='0.0.0.0', port=5000)

Public URL: NgrokTunnel: "https://2489-34-57-49-145.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 17:00:16] "GET /get_current_rules HTTP/1.1" 200 -


Received PDF: FR_Y-14Q20240331.pdf
No match found


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 17:04:18] "POST /generate_new_rules HTTP/1.1" 200 -


JSON data has been written to FRY14QH1_rules.json


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 17:04:22] "GET /get_current_rules HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 17:05:10] "POST /generate_anomalies_report HTTP/1.1" 200 -
